In [ ]:
import os
import shutil
from google.colab import drive

# Google 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers
!pip install -U datasets

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
!nvidia-smi

Sat Oct  5 11:57:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !conda install -U bitsandbytes
# !conda install -U accelerate

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'google/gemma-2-2b'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:

    device = 'cuda:0'

    prompt_template = """
    <start_of_turn>user
    Below is and instructionh that describes a task. Write a response that appropriately completes the request.
    {query}
    <end_of_turn>\n<start_of_turn>model


    """
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    # model = model.to("cuda:0")

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return decoded

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion.", model=model, tokenizer=tokenizer)
print(result)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



    user
    Below is and instructionh that describes a task. Write a response that appropriately completes the request.
    code the fibonacci series in python using reccursion.
    
model


    ```
```
    ```Python

```
    ```
    
    def fibonacci(n):
    if (n<2):
        return 1
    else:
        return (fibonacci(n-2) + fibonacci(n-1))
    return fibonacci(int(input(""))


    
    
    #python is an oop language. 
    #python3 
    #for python use

    #print a number
    #print the user input
     #enter and output in the console.
    #write a list
    #print list



In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.

    # if data_point['input']:
    #     text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

    # # Without

    # else:
    #     text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

    return text

In [ ]:
import pandas as pd

data = pd.read_csv('./law.csv')

data['prompt'] = data.apply(generate_prompt, axis=1)

data = data.sample(frac=1, random_state = 35).reset_index(drop=True)

data.head()

,instruction,output,prompt
0,방류수 수질 기준에 대한 내용으로 뭐가 있을까.,"공공하수처리시설ㆍ간이공공하수처리시설의 방류수수질기준은 별표 1과 같고, 분뇨처리시설...",<start_of_turn>user Below is an instruction th...
1,하수도 정비 중점 관리가 필요한 지역을 알려줘.,하수의 범람으로 인하여 인명ㆍ재산 등의 피해가 반복적으로 발생하거나 발생할 우려가 ...,<start_of_turn>user Below is an instruction th...
2,하수도 정비 지역을 변경할 때 장관의 역할은 뭐야.,"환경부장관은 중점관리지역을 지정ㆍ변경ㆍ해제하거나, 법 제4조의3제3항에 따라 특별시...",<start_of_turn>user Below is an instruction th...
3,하수관로 유지에 대한 조치를 알려줘.,공공하수도관리청은 매년 12월 말까지 해당 연도의 하수관로 유지관리 실적과 다음 연...,<start_of_turn>user Below is an instruction th...
4,하수 저류 시설의 종류에 대해 말해줘.,"침수예방을 위한 시설, 수질오염 저감 시설, 하수의 재이용을 위한 시설로 총 3가지...",<start_of_turn>user Below is an instruction th...


In [ ]:
tokenized_data = data['prompt'].apply(lambda x: tokenizer(x))

data['text'] = 'Below is and instructionh that describes a task. Write a response that appropriately completes the request.'
data['input_ids'] = tokenized_data.apply(lambda x: x['input_ids'])
data['attention_mask'] = tokenized_data.apply(lambda x: x['attention_mask'])

# data.drop(['tokenized_prompt'], axis = 1, inplace = True)

data.head()

,instruction,output,prompt,text,input_ids,attention_mask
0,방류수 수질 기준에 대한 내용으로 뭐가 있을까.,"공공하수처리시설ㆍ간이공공하수처리시설의 방류수수질기준은 별표 1과 같고, 분뇨처리시설...",<start_of_turn>user Below is an instruction th...,Below is and instructionh that describes a tas...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,하수도 정비 중점 관리가 필요한 지역을 알려줘.,하수의 범람으로 인하여 인명ㆍ재산 등의 피해가 반복적으로 발생하거나 발생할 우려가 ...,<start_of_turn>user Below is an instruction th...,Below is and instructionh that describes a tas...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,하수도 정비 지역을 변경할 때 장관의 역할은 뭐야.,"환경부장관은 중점관리지역을 지정ㆍ변경ㆍ해제하거나, 법 제4조의3제3항에 따라 특별시...",<start_of_turn>user Below is an instruction th...,Below is and instructionh that describes a tas...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,하수관로 유지에 대한 조치를 알려줘.,공공하수도관리청은 매년 12월 말까지 해당 연도의 하수관로 유지관리 실적과 다음 연...,<start_of_turn>user Below is an instruction th...,Below is and instructionh that describes a tas...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,하수 저류 시설의 종류에 대해 말해줘.,"침수예방을 위한 시설, 수질오염 저감 시설, 하수의 재이용을 위한 시설로 총 3가지...",<start_of_turn>user Below is an instruction th...,Below is and instructionh that describes a tas...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=35)

# print(f"Train size: {len(train_data)}")
# print(f"Test size: {len(test_data)}")

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'q_proj', 'k_proj', 'gate_proj', 'v_proj', 'up_proj', 'o_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 83066880 | total: 2697408768 | Percentage: 3.0795%


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

import transformers

from trl import SFTTrainer


tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        #warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

In [ ]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,3.470200
2,3.250000
3,2.115600
4,1.709800
5,1.321200
6,1.140600
7,0.890500
8,0.828900
9,0.552400
10,0.570200


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=100, training_loss=0.22181557927280665, metrics={'train_runtime': 903.4619, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.111, 'total_flos': 600029383449600.0, 'train_loss': 0.22181557927280665, 'epoch': 50.0})

In [ ]:
new_model = "gemma2-2B-ESG-Consultant" #Name of the model you will be pushing to huggingface model hub

trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)